In [13]:
import os
import glob
import requests
import datetime
from PIL import Image
import pandas as pd

## Load data

In [2]:
file_paths = glob.glob('all_industroes/raw/*')

In [5]:
file_df = pd.DataFrame()
for path in file_paths:
    try:
        df = pd.read_csv(path)
        if df.empty:
            continue
            
        file_df = file_df.append(df)
    except:
        pass

In [12]:
file_df.dtypes

industry_type            object
account_id                int64
img_url                  object
gender                   object
age                      object
impressions               int64
clicks                    int64
start_date       datetime64[ns]
end_date         datetime64[ns]
dtype: object

In [9]:
file_df.head()

,industry_type,account_id,img_url,gender,age,impressions,clicks,start_date,end_date
0,F&B,495921728367023,https://scontent-nrt1-1.xx.fbcdn.net/v/t45.160...,female,25-34,115,5,2021-07-25,2021-08-23
1,F&B,495921728367023,https://scontent-nrt1-1.xx.fbcdn.net/v/t45.160...,male,25-34,29,1,2021-07-25,2021-08-23
2,F&B,495921728367023,https://scontent-nrt1-1.xx.fbcdn.net/v/t45.160...,unknown,25-34,4,0,2021-07-25,2021-08-23
3,F&B,495921728367023,https://scontent-nrt1-1.xx.fbcdn.net/v/t45.160...,female,35-44,475,13,2021-07-25,2021-08-23
4,F&B,495921728367023,https://scontent-nrt1-1.xx.fbcdn.net/v/t45.160...,male,35-44,143,1,2021-07-25,2021-08-23


## FIlter out ongoing (not finished yet) records

In [11]:
file_df['start_date'] = pd.to_datetime(file_df['start_date'])
file_df['end_date'] = pd.to_datetime(file_df['end_date'])

In [16]:
date_until = datetime.date.today() - datetime.timedelta(1)

In [22]:
file_df = file_df.query("end_date <= '{}'".format(date_until))

In [23]:
len(file_df)

427732

## Regroup age labels

In [24]:
def adjust_age(val):
    if val in ['18-24', '25-34']:
        return '18-34'
    elif val in ['35-44', '45-54']:
        return '35-54'
    elif val in ['55-64', '65+']:
        return '55+'
    else:
        return val
    
file_df['age'] = file_df['age'].apply(adjust_age)

/Users/cheng-chunchang/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [26]:
file_df = file_df[file_df['age'].isin(['18-34', '35-54', '55+'])]

In [27]:
file_df = file_df[file_df['gender']!='unknown']

In [28]:
len(file_df)

348742

In [29]:
file_df.head()

,industry_type,account_id,img_url,gender,age,impressions,clicks,start_date,end_date
2587,EC,1162877107122120,https://scontent-nrt1-1.xx.fbcdn.net/v/t45.160...,female,18-34,57,2,2018-07-02,2018-09-26
2588,EC,1162877107122120,https://scontent-nrt1-1.xx.fbcdn.net/v/t45.160...,male,18-34,66,3,2018-07-02,2018-09-26
2589,EC,1162877107122120,https://scontent-nrt1-1.xx.fbcdn.net/v/t45.160...,female,35-54,195,7,2018-07-02,2018-09-26
2590,EC,1162877107122120,https://scontent-nrt1-1.xx.fbcdn.net/v/t45.160...,male,35-54,174,6,2018-07-02,2018-09-26
2591,EC,1162877107122120,https://scontent-nrt1-1.xx.fbcdn.net/v/t45.160...,female,35-54,632,23,2018-07-02,2018-09-26


## Calculate accumulative performance by segment

In [31]:
agg = file_df.groupby(['industry_type','account_id', 'img_url', 'age', 'gender', 'start_date', 'end_date']).sum()

In [32]:
agg.reset_index(inplace=True)

In [34]:
agg['flight'] = (agg['end_date'] - agg['start_date']).apply(lambda x: x.days)

In [36]:
def season_mapping(ser):
    timedelta = round(ser[1] / 2)
    month = (ser[0] + pd.Timedelta(timedelta, unit='D')).month
    if month in [1,2,3]:
        return 1
    elif month in [4,5,6]:
        return 2
    elif month in [7,8,9]:
        return 3
    elif month in [10,11,12]:
        return 4

agg['season'] = agg[['start_date', 'flight']].apply(season_mapping, axis=1)

In [38]:
agg['ctr'] = agg['clicks'] / agg['impressions'] * 100

In [64]:
len(agg)

90258

## Encode the urls to numbers for better usage later

In [39]:
img_url_list = agg['img_url'].unique()
n = 0
url_mapper = {}
for url in img_url_list:
    url_mapper[url] = str(n)
    n += 1

In [40]:
agg['url_encoded'] = agg['img_url'].map(url_mapper)

In [65]:
agg = agg.sample(frac=1, random_state=42)
agg['url_encoded'] = agg['url_encoded'].apply(lambda x: 'unique_images/' + str(x) + '.png')
agg['season'] = agg['season'].astype('str')
agg_revised = agg[(agg['ctr']!=0) & (agg['ctr']<=7)]

In [66]:
agg_revised.head()

,industry_type,account_id,img_url,age,gender,start_date,end_date,impressions,clicks,flight,season,ctr,url_encoded
70416,OTHERS,2620908031289149,https://scontent-nrt1-1.xx.fbcdn.net/v/t45.160...,35-54,male,2020-07-20,2020-10-28,182,6,100,3,3.296703,unique_images/8357.png
53290,H&B,338377303742414,https://scontent-nrt1-1.xx.fbcdn.net/v/t45.160...,35-54,female,2021-05-06,2021-06-07,2867,70,32,2,2.441577,unique_images/6504.png
33915,ENT,194852447853371,https://scontent-nrt1-1.xx.fbcdn.net/v/t45.160...,18-34,male,2019-09-30,2019-12-03,242016,5909,64,4,2.441574,unique_images/4712.png
5545,A&A,410133069812719,https://scontent-nrt1-1.xx.fbcdn.net/v/t45.160...,35-54,male,2020-08-25,2021-03-28,133,3,215,4,2.255639,unique_images/893.png
41381,F&B,197841007517765,https://scontent-nrt1-1.xx.fbcdn.net/v/t45.160...,55+,male,2020-11-04,2020-12-22,2050,2,48,4,0.097561,unique_images/5242.png


In [67]:
agg_revised.to_csv('all_industroes/agg_revised.csv', index=False)

In [42]:
agg.to_csv('all_industroes/agg.csv', index=False)

In [54]:
agg[agg['url_encoded']=='1813'].iloc[0]['img_url']

'https://scontent-nrt1-1.xx.fbcdn.net/v/t45.1600-4/14058496_6051961088752_2066266687_n.png?_nc_cat=102&ccb=1-3&_nc_sid=2aac32&_nc_ohc=HnwNzHZCqGgAX9jzOLv&_nc_ht=scontent-nrt1-1.xx&oh=09a6b296f0b79befc67de999f9295ef8&oe=61032A04'

## Download images

In [ ]:
n = 1
for url in img_url_list:
    try:
        res = requests.get(url, stream=True)
        file_name = 'all_industroes/unique_images/{}.png'.format(url_mapper[url])
        with open(file_name, 'wb') as f:
            f.write(res.content)
        print('image {} downloaded'.format(n))
        
    except:
        print('ERROR: {} {}'.format(url_mapper[url], url))
    
    finally:
        n += 1
    
    

## Check and convert image format, if not RGB 

In [58]:
img_path_list = glob.glob('all_industroes/unique_images/*')

In [62]:
for path in img_path_list:
    img = Image.open(path)
    if img.mode != 'RGB':
        print(path)
        img = img.convert('RGB')
        img.save(path)


all_industroes/unique_images/5492.png
all_industroes/unique_images/8175.png
all_industroes/unique_images/6158.png
all_industroes/unique_images/5684.png
all_industroes/unique_images/5135.png
all_industroes/unique_images/8559.png
all_industroes/unique_images/8203.png
all_industroes/unique_images/5727.png
all_industroes/unique_images/4439.png
all_industroes/unique_images/5519.png
all_industroes/unique_images/5525.png
all_industroes/unique_images/7333.png
all_industroes/unique_images/5524.png
all_industroes/unique_images/5518.png
all_industroes/unique_images/5726.png
all_industroes/unique_images/5134.png
all_industroes/unique_images/5685.png
all_industroes/unique_images/5450.png
all_industroes/unique_images/9242.png
all_industroes/unique_images/5478.png
all_industroes/unique_images/5493.png
all_industroes/unique_images/5491.png
all_industroes/unique_images/9240.png
all_industroes/unique_images/5452.png
all_industroes/unique_images/8176.png
all_industroes/unique_images/4758.png
all_industro

all_industroes/unique_images/6326.png
all_industroes/unique_images/6118.png
all_industroes/unique_images/3060.png
all_industroes/unique_images/7548.png
all_industroes/unique_images/4727.png
all_industroes/unique_images/4733.png
all_industroes/unique_images/330.png
all_industroes/unique_images/4723.png
all_industroes/unique_images/10173.png
all_industroes/unique_images/5763.png
all_industroes/unique_images/5777.png
all_industroes/unique_images/4469.png
all_industroes/unique_images/4680.png
all_industroes/unique_images/5561.png
all_industroes/unique_images/8051.png
all_industroes/unique_images/4125.png
all_industroes/unique_images/5560.png
all_industroes/unique_images/4871.png
all_industroes/unique_images/4468.png
all_industroes/unique_images/5158.png
all_industroes/unique_images/6337.png
all_industroes/unique_images/4508.png
all_industroes/unique_images/4722.png
all_industroes/unique_images/10172.png
all_industroes/unique_images/10166.png
all_industroes/unique_images/8867.png
all_indust